# Setup Flow

This notebook guides you through the setup flow of a new project.

### Variables
Set project environment. Default environment is `dev`.

In [2]:
env = "dev"
bucket_name = "api-terraform-state-dev"

Define basic project values.

In [3]:
project_id = "template-nestjs-api"
project_name = "BE Template API"

Define project secrets.

In [56]:
jwt_secret_value = "some-jwt-secret"
db_username = "some-db-user"
db_password = "some-db-password"

Define other variables.

In [5]:
region = "us-central1"
docker_image_name = "app-service"
db_tier = "db-f1-micro"

Create `tfvars` file for the `environments` folder.

In [6]:
import os

# Path to the terraform.tfvars file
tfvars_file_path = f"environments/{env}/terraform.tfvars"

# Check if the file exists
if os.path.isfile(tfvars_file_path):
  # If the file exists, delete it
  os.remove(tfvars_file_path)
  print(f"Existing terraform.tfvars file deleted: {tfvars_file_path}")

# Variables for the terraform.tfvars file content
terraform_tfvars_content = f"""environment = "{env}"
project_id = "{project_id}"
region = "{region}"

# App
app_api_prefix = "api"
app_api_name = "{project_name} [{env}]"
app_api_auth_jwt_token_expires_in = "1d"

# Database
database_name = "api_db"
db_tier= "{db_tier}"
"""

# Write content to the terraform.tfvars file
with open(tfvars_file_path, "w") as f:
  f.write(terraform_tfvars_content)

print(f"terraform.tfvars file created at: {tfvars_file_path}")

Existing terraform.tfvars file deleted: environments/dev/terraform.tfvars
terraform.tfvars file created at: environments/dev/terraform.tfvars


## Change CHDIR

We need to change the current working directory to the `environments/{env}` folder so that terraform commands can be executed in the correct directory.

In [7]:
import os
os.chdir(f"./environments/{env}")
print("Current working directory:", os.getcwd())

Current working directory: /Users/michaljarnot/IdeaProjects/backend-template-nestjs-api-5/infra/environments/dev


## Google Cloud Project Setup

In [9]:
# Authenticate with Google Cloud
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=bk91M77u6ZFxrpqTvP2bHaEbdTvrJH&access_type=offline&code_challenge=WbIVVzy63dpFzmjOO0ChJl6RARhQdocO8Czb-wbY7Ag&code_challenge_method=S256


You are now logged in as [m.jarnot@yahoo.com].
Your current project is [some-project-id-546757].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



In [10]:
# Create a new Google Cloud project
!gcloud projects create {project_id} --name="{project_name}"

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/template-nestjs-api].
Waiting for [operations/cp.8786503144929325054] to finish...done.              
Enabling service [cloudapis.googleapis.com] on project [template-nestjs-api]...
Operation "operations/acat.p2-812254980684-71f75b6d-6268-4327-a351-6735ca6a09c5" finished successfully.


In [11]:
# Set your Google Cloud project
!gcloud config set project {project_id}

Updated property [core/project].


In [12]:
# Set the default region and zone for your project
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=icJlWLZn8tbArzWU0RbE3hOPnOIZBe&access_type=offline&code_challenge=Xug4iIwabi_PhAeJw_LtmnIRYoZMegDqLBm3jSVcZs0&code_challenge_method=S256


Credentials saved to file: [/Users/michaljarnot/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "template-nestjs-api" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project own

## Required Google Cloud Services
Link billing account to your project.

In [13]:
import webbrowser
billing_account_link = "https://console.cloud.google.com/billing/linkedaccount?hl=en&project=" + project_id
webbrowser.open(billing_account_link)

True

In [14]:
!gcloud services enable artifactregistry.googleapis.com compute.googleapis.com cloudbuild.googleapis.com secretmanager.googleapis.com sqladmin.googleapis.com storage.googleapis.com iam.googleapis.com cloudresourcemanager.googleapis.com run.googleapis.com

Operation "operations/acf.p2-812254980684-e9c4c75f-0522-42af-ac4d-f50de691f4bd" finished successfully.


## GCS Bucket for Terraform State:

In [15]:
# Create a GCS Bucket
!gsutil mb -l {region} gs://{bucket_name}/

Creating gs://api-terraform-state-dev/...


In [16]:
# Enable Versioning
!gsutil versioning set on gs://{bucket_name}/

Enabling versioning for gs://api-terraform-state-dev/...


## Secrets Manager

In [17]:
# JWT Secret in Google Secret Manager
!gcloud secrets create {env}-auth-jwt-secret --replication-policy="automatic"
!echo -n {jwt_secret_value} | gcloud secrets versions add {env}-auth-jwt-secret --data-file=-

Created secret [dev-auth-jwt-secret].
Created version [1] of the secret [dev-auth-jwt-secret].


In [18]:
# Database Credentials Secrets
!gcloud secrets create {env}-db-username --replication-policy="automatic"
!echo -n {db_username} | gcloud secrets versions add {env}-db-username --data-file=-

!gcloud secrets create {env}-db-password --replication-policy="automatic"
!echo -n {db_password} | gcloud secrets versions add {env}-db-password --data-file=-

Created secret [dev-db-username].
Created version [1] of the secret [dev-db-username].
Created secret [dev-db-password].
Created version [1] of the secret [dev-db-password].


## Terraform Deployment:

Initialize and apply your Terraform configuration as usual. Include the path to `tfvars`.

**Initialize Terraform**

In [19]:
# Initialize Terraform
!terraform init


Initializing the backend...

Successfully configured the backend "gcs"! Terraform will automatically
use this backend unless the backend configuration changes.
Initializing modules...
- artifact_registry in ../../modules/artifact_registry
- ci_cd_service_account in ../../modules/ci_cd_service_account
- cloud_run in ../../modules/cloud_run
- cloud_sql in ../../modules/cloud_sql
- load_balancer in ../../modules/load_balancer

Initializing provider plugins...
- Finding hashicorp/google versions matching "5.19.0"...
- Installing hashicorp/google v5.19.0...
- Installed hashicorp/google v5.19.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraf

**Apply Terraform changes**

Note: The following command will fail with an error because we haven't uploaded Docker image yet. `│ Error: Error waiting to create Service: resource is in failed state "Ready:False", message: Revision 'some-project-id-546757-dev-service-00001-6sm' is not ready and cannot serve traffic. Image 'us-central1-docker.pkg.dev/some-project-id-546757/some-project-id-546757-dev-repo/app-service:latest' not found.`

In [21]:
# Execute Terraform apply directly with hardcoded app_service_count and dynamic tfvars_file_path
!terraform apply --auto-approve

Acquiring state lock. This may take a few moments...
module.cloud_sql.data.google_secret_manager_secret_version.db_username: Reading...
module.cloud_sql.data.google_secret_manager_secret_version.db_password: Reading...
module.cloud_sql.data.google_compute_default_service_account.default: Reading...
module.cloud_run.data.google_compute_default_service_account.default: Reading...
module.cloud_sql.data.google_secret_manager_secret_version.db_password: Read complete after 1s [id=projects/812254980684/secrets/dev-db-password/versions/1]
module.cloud_sql.data.google_secret_manager_secret_version.db_username: Read complete after 1s [id=projects/812254980684/secrets/dev-db-username/versions/1]
module.cloud_run.data.google_compute_default_service_account.default: Read complete after 1s [id=projects/template-nestjs-api/serviceAccounts/812254980684-compute@developer.gserviceaccount.com]
module.cloud_sql.data.google_compute_default_service_account.default: Read complete after 1s [id=projects/templ

## Docker Image Build & Deployment

In [22]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "asia.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud",
    "us.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


In [23]:
# Build the Docker image using the Dockerfile located in the parent directory.
!docker build -t {region}-docker.pkg.dev/{project_id}/{project_id}-{env}-repo/{env}-{docker_image_name}:latest ../../../

[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (2/3)                                    docker:desktop-linux
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 311B                                          0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 493B                                       0.0s
 => [internal] load metadata for docker.io/library/node:18-alpine3.16      0.2s
[+] Building 0.3s (2/3)                                    docker:desktop-linux
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 311B                                          0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 493B                                       0.0s
 => [internal] load metadata for docker.

In [24]:
# Push Docker Image to Artifact Registry
!docker push {region}-docker.pkg.dev/{project_id}/{project_id}-{env}-repo/{env}-{docker_image_name}:latest

The push refers to repository [us-central1-docker.pkg.dev/template-nestjs-api/template-nestjs-api-dev-repo/dev-app-service]

4fd2ff21: Preparing 
927df009: Preparing 
21e1ee48: Preparing 
a00b35e9: Preparing 
32cce104: Preparing 
59b2cb08: Preparing 
6cdc78a2: Preparing 
927df009: Pushed   1.518GB/1.475GBlatest: digest: sha256:dbae0dc8150b4b0f9569a1b6ed900ef27065b45489d268abaa24ab1aa4642795 size: 1996


In [25]:
# Deploy to Google Cloud Run
!gcloud run deploy {project_id}-{env}-service --image {region}-docker.pkg.dev/{project_id}/{project_id}-{env}-repo/{env}-{docker_image_name}:latest --platform managed --region={region} --allow-unauthenticated

Deploying container to Cloud Run service [template-nestjs-api-dev-service] in project [template-nestjs-api] region [us-central1]
Deploying...                                                                   
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      
  Deploying...                                                                 



⠛ Deploying...                                                                 



⠹ Deploying...                                                                 



⠼ Deploying...                                                                 



⠶ Deploying...                                                                 



⠧ Deploying...                                                                 


  ⠧ Setting IAM Policy...                             

## Secrets for CI Authorization

In [26]:
# Define your variables
service_account_name = f"{env}-ci-cd-service-account"
service_account_email = f"{service_account_name}@{project_id}.iam.gserviceaccount.com"
key_filename = f"{env}-ci-cd-service-account-key.json"

# Construct the gcloud command
gcloud_command = f"gcloud iam service-accounts keys create ../../../{key_filename} --iam-account {service_account_email}"

# Execute the command
!{gcloud_command}

created key [1af2b81d7c7fa9add606cfe5e2766dc29621cdf9] of type [json] as [../../../dev-ci-cd-service-account-key.json] for [dev-ci-cd-service-account@template-nestjs-api.iam.gserviceaccount.com]


## GitHub Secrets

Go to your GitHub repository and navigate to `Settings` > `Secrets`.

Add a new secrets:

- GCP_PROJECT_ID
- GCP_REGION
- GCP_{env}_SA_KEY
- GCP_{env}_REPOSITORY
- GCP_{env}_IMAGE_NAME

In [27]:
# Print secrets for GitHub
print(f"GCP_PROJECT_ID: {project_id}")
print(f"GCP_REGION: {region}")
print(f"GCP_{env.upper()}_SA_KEY: >>contents of {key_filename}<<")
print(f"GCP_{env.upper()}_REPOSITORY: {project_id}-{env}-repo")
print(f"GCP_{env.upper()}_IMAGE_NAME: {env}-{docker_image_name}")

GCP_PROJECT_ID: template-nestjs-api
GCP_REGION: us-central1
GCP_DEV_SA_KEY: >>contents of dev-ci-cd-service-account-key.json<<
GCP_DEV_REPOSITORY: template-nestjs-api-dev-repo
GCP_DEV_IMAGE_NAME: dev-app-service
